In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import openai
import os

In [ ]:


def load_data():
    file_path = "/content/drive/MyDrive/Colab Notebooks/DATA_VISUALIZATION/dislit_final.csv"
    df = pd.read_csv(file_path, header=0, sep=',')
    df["DataCriacao1"] = pd.to_datetime(df["DataCriacao1"], format="%d/%m/%Y")
    df['month'] =df["DataCriacao1"].dt.strftime('%b')
    df['year'] = df["DataCriacao1"].dt.year.astype(str)
    df['year_month'] = df["DataCriacao1"].dt.to_period("M").astype(str)
    df = df.drop(columns=['DataCriacao1'])
    return df

df = load_data()

In [5]:
print(df.columns)

Index(['Textbox2', 'NumPregão2', 'NumParticipantes', 'Orgao2', 'SIGLA',
       'Modalidade2', 'Data de Abertura', 'Objeto', 'ClassObjeto',
       'Razão Social', 'Total Adjudicado', 'Valor', 'month', 'year',
       'year_month'],
      dtype='object')


## DICIONARIO PARA PADRONIZAÇÃO DOS ÓRGAOS PELAS SIGLAS

In [10]:
#DICIONARIO PARA PADRONIZAÇÃO DOS ÓRGAOS PELAS SIGLAS
siglas = {'SEDH':[
 'SEDH/FUND. DESENV. CRIANÇA E ADOLESC"ALICE DE ALMEIDA"',
 'SEDH/LOTERIA DO ESTADO DA PARAÍBA' 'CCG/PARAÍBA PREVIDÊNCIA',
 'SEDH/FUNDO ESTADUAL DE ASSISTÊNCIA SOCIAL',
 'SEDH/FUNDAÇÃO DE DESENVOLVIMENTO DA CRIANÇA E DO ADOLESCENTE "ALICE DE ALMEIDA"',
 'SEDH/COMPANHIA ESTADUAL DE HABITAÇÃO POPULAR',  'SECRETARIA DE ESTADO DE DESENVOLVIMENTO HUMANO'],
 'SES':[
 'SES/COMPLEXO DE SAÚDE ARLINDA MARQUES' ,
 'SES/AGÊNCIA ESTADUAL DE VIGILÂNCIA SANITÁRIA',
 'SES/COMPLEXO DE SAÚDE CLEMENTINO FRAGA',
 'SES/HOSPITAL DE TRAUMAS DE JOÃO PESSOA',
 'SES/COMPLEXO DE SAÚDE DE CAJAZEIRAS',
 'SECRETARIA DE ESTADO DA SAÚDE',
 'SES/HOSPITAL REGIONAL DE PATOS',
 'SES/HOSPITAL GERAL DE MAMANGUAPE',
 'SES/HOSPITAL METROPOLITANO DOM JOSE MARIA PIRES - SANTA RITA',
 'SES/UNIDADE DE PRONTO ATENDIMENTO CAJAZEIRAS - UPA',
 'SES/HOSPITAL DISTRITAL DE POMBAL',
 'SES/COMPLEXO DE SAÚDE JULIANO MOREIRA',
 'SES/COMPLEXO DE SAÚDE DE GUARABIRA',
 'SES/COMPLEXO DE SAÚDE DE CRUZ DAS ARMAS',
 'SES/MATERNIDADE DR. PEREGRINO FILHO (PATOS)',
 'SES/CENTRO ESPECIALIZADO EM REABILITACAO - CER IV',
 'SES/HOSPITAL DISTRITAL DE SOLÂNEA - SOLANEA',
 'SES/FUNDACAO PARAIBANA DE GESTAO EM SAUDE -PB SAUDE',
 'SES/UNIDADE DE PRONTO ATENDIMENTO UPA 24 HORAS DO MUNICÍPIO DE PRINCESA ISABEL',
 'SES/HOSPITAL GERAL DE COREMAS',
 'SES/HOSPITAL DR AMERICO MAIS VASCONCELOS - CATOLE DO ROCHA',
 'SES/CONSELHO ESTADUAL DE SAUDE',
 'SES/COMPLEXO DE SAÚDE DE PICUÍ',
 'SES/HOSPITAL REGIONAL WENCESLAU LOPES - PIANCO',
 'SES/HOSPITAL DE TRAUMAS DE CAMPINA GRANDE',
 'SES/HOSPITAL GERAL DE QUEIMADAS - QUEIMADAS',
 'SES/UNIDADE DE PRONTO ATENDIMENTO GUARABIRA - UPA',
 'SES/UNIDADE DE PRONTO ATENDIMENTO SANTA RITA - UPA',
 'SES/HOSPITAL DE CLINICAS DE CAMPINA GRANDE',
 'INSTITUTO HOSPITALAR GENERAL EDSON RAMALHO',
 'LACEN - LABORATORIO CENTRAL DE SAÚDE PÚBLICA',
 'CENTRO ESPECIALIZADO DIAGNÓSTICO DO CÂNCER'
],
'SEAFDS': ['SEAFDS'],
'PMPB': ['POLÍCIA MILITAR DO ESTADO DA PARAÍBA'],
'BMPB': ['CORPO DE BOMBEIROS MILITAR DA PARAÍBA'],
'SEE':['SECRETARIA DE ESTADO DA EDUCAÇÃO E DA CIÊNCIA E TECNOLOGIA',
 'SECRETARIA DE ESTADO DA EDUCAÇÃO','SEE', 'SEE/FUNDAÇÃO CENTRO INTEGRADO DE APOIO AO PORTADOR DE DEFICIÊNCIA'],
'SECTMA': ['SECTMA/FUNDAÇÃO DE APOIO À PESQUISA DO ESTADO DA PARAÍBA',
 'SECTMA/SUPERINTENDÊNCIA DE ADMINISTRAÇÃO DO MEIO AMBIENTE',
 'SECTMA',
 'SECTMA/FUNDO ESTADUAL DE RECURSOS HÍDRICOS',
 'SECTMA/AGÊNCIA EXECUTIVA DE GESTÃO DAS ÁGUAS DO ESTADO DA PARAÍBA'
],
'SEFAZ': ['SEFAZ','SEFAZ/LOTERIA DO ESTADO DA PARAÍBA'],
'SECOM': ['SECOM/EMPRESA PARAIBANA DE COMUNICAÇÃO S/A', 'SECOM/A UNIÃO SUPERINTENDÊNCIA DE IMPRENSA E EDITORA', 'SECRETARIA DE ESTADO DA COMUNICAÇÃO INSTITUCIONAL'],
'PGE':['PROCURADORIA GERAL DO ESTADO'],
'SER':['SECRETARIA DE ESTADO DA RECEITA'],
'EGE': ['ENCARGOS GERAIS DO ESTADO'],
'SEIE':['SEIE/COMPANHIA DOCAS DA PARAÍBA'],
'SEAD':['SECRETARIA DE ESTADO DA ADMINISTRAÇÃO', 'SEAD/FUNDO ESPECIAL DE DESENVOLVIMENTO DE RECURSOS HUMANOS'],
'SEC': ['SEC','SECRETARIA DE ESTADO DA CULTURA','SEC/INSTITUTO DO PATRIMÔNIO HISTÓRICO E ARTÍSTICO DO ESTADO DA PARAÍBA'],
'CCG': ['CCG/AGÊNCIA DE REGULAÇÃO DO ESTADO DA PARAÍBA','CONTROLADORIA GERAL DO ESTADO'],
'SEPLAN':['SECRETARIA DE ESTADO DE PLANEJAMENTO', 'SECRETARIA DE ESTADO DO PLANEJAMENTO'],
'SEDAP':['SECRETARIA DE ESTADO DO DESENVOLVIMENTO DA AGROPECUÁRIA E DA PESCA', 'SEDAP/FUNDO DE DESENVOLVIMENTO AGROPECUÁRIO DO ESTADO DA PARAÍBA'],
'DETRAN':['SEDS/DEPARTAMENTO ESTADUAL DE TRÂNSITO DO ESTADO DA PARAÍBA'],
'POLICIA CIVIL': ['POLICIA CIVIL DO ESTADO DA PARAIBA'],
'CASA MILITAR':['CASA MILITAR'],
 'SETDE':['SETDE', 'SECRETARIA DE ESTADO DO TURISMO E DO DESENVOLVIMENTO ECONÔMICO',
 'SETDE/INSTITUTO DE METROLOGIA E QUALIDADE INDUSTRIAL DA PARAÍBA',
 'SETDE/JUNTA COMERCIAL DO ESTADO DA PARAIBA'],
'SEINFRA': ['SECRETARIA DE ESTADO DA INFRAESTRUTURA'],
'SEIE': ['SEIE/SUPERINTENDÊNCIA DE OBRAS DO PLANO DE DESENVOLVIMENTO DO ESTADO',
'SEIE/DEPARTAMENTO DE ESTRADAS DE RODAGEM'],
'SESDS':[ 'SECRETARIA DE ESTADO DA SEGURANÇA E DA DEFESA SOCIAL'],
'SEAP':['SECRETARIA DE ESTADO DA ADMINISTRAÇÃO PENITENCIÁRIA'],
 'PROJETO COOPERAR': ['PROJETO COOPERAR'] ,
 'PROJETO CARIRI':['PROJETO DE DESENVOLVIMENTO SUSTENTÁVEL DO CARIRI'],
'IPEP':['INSTITUTO DE ASSISTÊNCIA À SAÚDE DO SERVIDOR'],
'SEJEL':['SECRETARIA DE ESTADO DA JUVENTUDE','FUNDO DE APOIO AO ESPORTE E LAZER DO ESTADO DA PARAÍBA'],
 'FEPDC':['FUNDO ESTADUAL DE DEFESA DOS DIREITOS DO CONSUMIDOR'],
 'SEMDH':['SECRETARIA DE ESTADO DA MULHER E DA DIVERSIDADE HUMANA'],
 'SEG':['SECRETARIA DE ESTADO DO GOVERNO'],
 'SEDAM':['SECRETARIA DE ESTADO DO DESENVOLVIMENTO E DA ARTICULAÇÃO MUNICIPAL']
}

In [12]:
# Criar um mapeamento inverso (valores para chaves)
mapeamento = {orgao: sigla for sigla, orgaos in siglas.items() for orgao in orgaos}

# Aplicar ao DataFrame
df['Siglas'] = df['Orgao2'].map(mapeamento)

# Exibir resultado
print(df[['Orgao2', 'Siglas']])

                                                 Orgao2 Siglas
0     SEDH/FUND. DESENV. CRIANÇA E ADOLESC"ALICE DE ...   SEDH
1     SEDH/FUND. DESENV. CRIANÇA E ADOLESC"ALICE DE ...   SEDH
2                       SECRETARIA DE ESTADO DA RECEITA    SER
3                             ENCARGOS GERAIS DO ESTADO    EGE
4                 SES/COMPLEXO DE SAÚDE ARLINDA MARQUES    SES
...                                                 ...    ...
5613              SES/COMPLEXO DE SAÚDE ARLINDA MARQUES    SES
5614                      SECRETARIA DE ESTADO DA SAÚDE    SES
5615              SES/COMPLEXO DE SAÚDE ARLINDA MARQUES    SES
5616              SES/COMPLEXO DE SAÚDE ARLINDA MARQUES    SES
5617              SES/COMPLEXO DE SAÚDE ARLINDA MARQUES    SES

[5618 rows x 2 columns]


#PADRONIZAÇÃO DOS OBJETOS ATRAVÉS DO AGENTE DE IA

In [ ]:
os.environ["OPENAI_API_KEY"] =  ""

# Usar a variável de ambiente
openai.api_key = os.getenv("OPENAI_API_KEY")


def classificar_frase(frase):
    prompt = f"""
    Classifique a seguinte frase em uma das categorias:  Obras e serviços de engenharia,     Materiais de Expediente,     Compra de Medicamentos,
    Material e Equipamentos Hospitalares,     Exame Hospitalar,     Alimentação Hospitalar,  Suplementos e Leito Fórmula  Serviços de telefonia e internet,
    Serviços de TI,     Contratação de serviços em geral,     Alimentação para Eventos,     Alimentação para Animais,    Alimentação para quentinhas e outros,
    Serviços Gráficos e de Impressão,    Locação de Imóvel,    Conserto e Manutenção de equipamentos,    Compra de material permanente



    Frase: "{frase}"

    Retorne apenas o nome da categoria.
    """

    resposta = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # Ou outro modelo
        messages=[{"role": "user", "content": prompt}]
    )

    return resposta["choices"][0]["message"]["content"].strip()



In [ ]:
for i in range(len(df)):
  objeto = df.loc[i, 'Modalidade2']
  classe = classificar_frase(objeto)
  df.loc[i, 'ClasseObj'] = classe

In [ ]:
save_file = '/content/drive/MyDrive/Colab Notebooks/DATA_VISUALIZATION/df_padronizado.csv'
df.to_csv(save_file, encoding="latin1")